## Setup

In [2]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!git clone https://github.com/invi-bhagyesh/qllm-eval-1.git
%cd qllm-eval-1
!pip install -e .
!pip install transformers -U
!pip install lm-eval --upgrade
!pip install trl fastchat

Cloning into 'qllm-eval-1'...
remote: Enumerating objects: 601, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 601 (delta 39), reused 34 (delta 19), pack-reused 545 (from 1)
Receiving objects: 100% (601/601), 685.84 KiB | 29.82 MiB/s, done.
Resolving deltas: 100% (352/352), done.


# Single run RLHF

In [ ]:
models = [
    "stabilityai/stablelm-2-1_6b",
    "microsoft/phi-1_5",
    "HuggingFaceTB/SmolLM-1.7B",
    "Qwen/Qwen2.5-1.5B",
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
]

tasks = [
    "crows_pairs_english_race_color",
    "crows_pairs_english_gender",
    "crows_pairs_english_religion"
]

w_bits = [16, 8, 4, 2]

# Pull latest repo and switch to rl branch
!git pull
!git checkout rl

for model in models:
    for task in tasks:
        for w_bit in w_bits:
            print(f"Running model: {model}, task: {task}, w_bit: {w_bit}")

            if w_bit == 16:
                !CUDA_VISIBLE_DEVICES=0 python qllm_eval/evaluation/q_harness/main.py \
                    --model_path "{model}" \
                    --tasks "{task}" \
                    --w_bit 16 \
                    --a_bit 16 \
                    --w_group_size 128 \
                    --kv_bit 16 \
                    --dataset crows-pairs
            else:
                !CUDA_VISIBLE_DEVICES=0 python qllm_eval/evaluation/q_harness/main.py \
                    --model_path "{model}" \
                    --tasks "{task}" \
                    --w_bit {w_bit} \
                    --a_bit 16 \
                    --w_group_size 128 \
                    --kv_bit 16 \
                    --dataset crows-pairs \
                    --dpo_path "/content/qllm-eval-1/dpo_finetuned_{task}"

        print(f"\n===== Results for {task} with model {model} =====")
        for w_bit in w_bits:
            result_path = f"/content/qllm-eval-1/{task}/{model}/kv_16_w_{w_bit}_a_16.jsonl"
            print(f"\n==> Showing: {result_path}")
            !cat "{result_path}"


Already up to date.
Already on 'rl'
Your branch is up to date with 'origin/rl'.
Running model: microsoft/phi-1_5, task: crows_pairs_english_race_color, w_bit: 16
2025-11-07 15:04:41.778862: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-07 15:04:41.796443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762527881.817957   21293 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762527881.824482   21293 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea